In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/processed-ontonotes5.json

In [2]:
import json
from sklearn.model_selection import train_test_split
import random

In [3]:
with open('processed-ontonotes5.json') as fopen:
    data = json.load(fopen)

In [4]:
from collections import defaultdict

entities = defaultdict(list)
for i in data:
    entities['text'].append(i[0])
    entities['label'].append(i[1])

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/classic-song.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/filem-malaysia.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/filem-pramlee.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/lagu-lirikterbaikz.blogspot.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/lagu-pramlee.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/WORK_OF_ART/novel.txt

In [6]:
with open('classic-song.txt') as fopen:
    data = fopen.read().split('\n')
    
classic_song = [i.split('\t')[1].strip() for i in data if len(i)]

In [7]:
with open('filem-malaysia.txt') as fopen:
    data = fopen.read().split('\n')
    
filem_malaysia = [i.split('\t')[0].strip() for i in data if len(i)][:43]
filem_malaysia.extend([i.split('\t')[1].strip() for i in data if len(i)][43:])

In [8]:
with open('filem-pramlee.txt') as fopen:
    data = fopen.read().split('\n')
    
filem_pramlee = [i.split('\t')[0].strip() for i in data if len(i)]

In [9]:
with open('lagu-lirikterbaikz.blogspot.txt') as fopen:
    lagu = fopen.read().split('\n')

In [10]:
with open('novel.txt') as fopen:
    data = fopen.read().split('\n')[1:-1]
    
buku = []
for i in range(1, len(data), 8):
    buku.append(data[i].split('\t')[0].strip())

In [11]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'WORK_OF_ART':
        while entities['label'][i] == 'WORK_OF_ART':
            r.append(i)
            i += 1
        results.append(r)
    i += 1
    
len(results)

1457

In [12]:
art = classic_song + filem_malaysia + filem_pramlee + lagu + buku

In [13]:
import math
import numpy as np

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [14]:
train_results, test_results = train_test_split(results, test_size = 0.2)
train_art, test_art = train_test_split(art, test_size = 0.2)

In [15]:
train_X, train_Y = [], []

repeat = 15
for t in train_art:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

4020

In [16]:
test_X, test_Y = [], []

repeat = 15
for t in test_art:
    for i in range(repeat):
        x, y = generate_index(test_results[random.randint(0, len(test_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        test_X.append(x)
        test_Y.append(y)
            
len(test_X)

1020

In [17]:
len(train_X), len(test_X)

(4020, 1020)

In [18]:
with open('augmentation-work-of-art-ontonotes5.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y,
              'test_X': test_X, 'test_Y': test_Y}, fopen)